In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,roc_curve,auc,classification_report,make_scorer
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import  VotingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt

In [ ]:
from link_prediction import *

In [ ]:
#Load the entity embeddings dict
with open("dbpedia50_distmult_entity_embeddings.pkl", "rb") as f:
    entity_embeddings = pickle.load(f)
#Load the predicate embeddings dict
with open("dbpedia50_distmult_predicate_embeddings.pkl", "rb") as f:
    predicate_embeddings = pickle.load(f)

In [ ]:
train_triples = pd.read_csv('dbpedia50_train.csv', dtype=str)
#val_triples = pd.read_csv('dbpedia50_valid.csv',dtype=str)
test_triples = pd.read_csv('dbpedia50_test_filtered.csv', dtype=str)

In [ ]:
train_triples['label']=1

In [ ]:
# Read the CSV files
head_df = pd.read_csv('dbpedia50_distmult_head_df_1_ns.csv')
#head_df = head_df.drop(index=0).reset_index(drop=True)  # Resetting the index after dropping
tail_df = pd.read_csv('dbpedia50_distmult_tail_df_1_ns.csv')
#tail_df = tail_df.drop(index=0).reset_index(drop=True)  # Resetting the index after dropping
# Combine the datasets by concatenating them along rows (axis=0)
train_df = pd.concat([head_df,tail_df,train_triples], axis=0)
# Reset the index
train_df.reset_index(drop=True, inplace=True)
# Drop duplicate rows
train_df.drop_duplicates(inplace=True)

In [ ]:
train_df.label.value_counts()

,count
label,
0,40014
1,32203


In [ ]:
tripleEvaluator=TripleEvaluator(entity_embeddings,predicate_embeddings,train_triples,test_triples, model='DistMult', k=50)

Precomputing top-k predictions: 100%|██████████| 2098/2098 [09:09<00:00,  3.82it/s]


In [ ]:
# Step 1: Prepare the training data
train_df['embedding'] = train_df.apply(lambda row: tripleEvaluator.get_embedding(row), axis=1)
X_train = np.vstack(train_df['embedding'].values)  # Stack embeddings into a matrix
y_train = train_df['label'].values  # Labels

In [ ]:
# Step 1: Split the data into training and test sets with stratification
X_train, X_test, y_train, y_test = train_test_split(
    np.vstack(train_df['embedding'].values),  # Embeddings matrix
    train_df['label'].values,                # Labels
    test_size=0.1,                           # 10% of the data for testing
    random_state=42,                         # For reproducibility
    stratify=train_df['label'].values        # Maintain label distribution
)

In [ ]:
# Step 1: Initialize the MLP model
mlp_model = MLPClassifier(
    hidden_layer_sizes=(128, 128),  # Two layers with k units each
    activation='relu',              # ReLU activation function
    solver='adam',                  # Adam optimizer
    alpha=0.0001,                   # L2 regularization
    learning_rate_init=0.001,       # Learning rate
    early_stopping=True,            # Stops if validation score does not improve
    max_iter=25,                     # Number of iterations
    batch_size=32,                  # Same batch size
    random_state=42                 # For reproducibility
)
mlp_model.fit(X_train, y_train)

# Step 4: Evaluate the model on the test set
y_pred = mlp_model.predict(X_test)

# Print evaluation metrics
print("Accuracy on test set:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

In [ ]:
# Create an XGBoost model 
xgb_model = XGBClassifier(
    #scale_pos_weight=2,    # Adjust the weight for the positive class (useful for imbalance)
    n_estimators=100,       # Number of boosting rounds
    max_depth=10,           # Maximum depth of a tree
    #min_child_weight=,
    #subsample=,
    #colsample_bytree=
    random_state=42         # Set a random state for reproducibility
)

# Fit the model
xgb_model.fit(X_train, y_train)

# Make predictions
y_pred_xgb = xgb_model.predict(X_test)

# Print evaluation metrics
print("XGBoost Classification Report:\n", classification_report(y_test, y_pred_xgb))

In [ ]:
# Create a LightGBM model
lgbm_model = LGBMClassifier(
    #class_weight={0: 1, 1: 2}, # To handle imbalanced data
    n_estimators=100,        # Number of boosting rounds
    max_depth=15,            # Maximum depth of a tree
    #num_leaves= 100,              # Maximum number of leaves in one tree
    #min_child_samples=25 ,       # Minimum data per leaf
    #subsample= 1,
    random_state=42          # Set a random state for reproducibility
)
# Fit the model
lgbm_model.fit(X_train, y_train)

# Make predictions
y_pred_lgbm = lgbm_model.predict(X_test)

# Print evaluation metrics
print("LightGBM Classification Report:\n", classification_report(y_test, y_pred_lgbm))

In [ ]:
# Create the voting classifier
ensemble_model = VotingClassifier(
    estimators=[
        ('xgb', xgb_model),
        ('mlp', mlp_model),
        ('lgbm', lgbm_model)
    ],
    voting='soft',  # Soft voting to use probabilities
    n_jobs=-1       # Use all cores for parallel processing
)

# Fit the ensemble model (if needed)
ensemble_model.fit(X_train, y_train)

# Evaluate the ensemble
y_pred_ensemble = ensemble_model.predict(X_test)

print("Ensemble Accuracy:", accuracy_score(y_test, y_pred_ensemble))
print("Classification Report for Ensemble:\n", classification_report(y_test, y_pred_ensemble))

**ORIGINIAL**

In [ ]:
results = {}

# Evaluate for k from 1 to 10 and store the results in a dictionary
for k in [1, 3, 5, 10, 20, 30, 40, 50, 100]:
    total_count_head, hit_count_head = tripleEvaluator.evaluate_hitk_original_2_on_patterns(k=k, evaluate_for="head")
    total_count_tail, hit_count_tail = tripleEvaluator.evaluate_hitk_original_2_on_patterns(k=k, evaluate_for="tail")

    overall_hit_count = hit_count_head + hit_count_tail
    overall_total_count = total_count_head + total_count_tail
    overall_percentage = (overall_hit_count / overall_total_count) * 100

    results[k] = {
        "head": {
            "total_count": len(tripleEvaluator.precomputed_top_k_head),
            "hit_count": hit_count_head,
            "percentage": hit_count_head / total_count_head * 100
        },
        "tail": {
            "total_count": len(tripleEvaluator.precomputed_top_k_tail),
            "hit_count": hit_count_tail,
            "percentage": hit_count_tail / total_count_tail * 100
        },
        "overall": {
            "hit_count": overall_hit_count,
            "total_count": overall_total_count,
            "percentage": overall_percentage
        }
    }

# Print the results nicely
for k, data in results.items():
    print(
        f"k={k}: "
        f"Head: {data['head']['percentage']:.2f}% | "
        f"Tail: {data['tail']['percentage']:.2f}% | "
        f"Overall: {data['overall']['percentage']:.2f}%"
    )

**MLP**

In [ ]:
results = []

# Define the mapping of k values to their respective threshold lists
k_threshold_mapping = {
    1: [1, 3, 5, 10, 20, 30, 40, 50],
    3: [3, 5, 10, 20, 30, 40, 50],
    5: [5, 10, 20, 30, 40, 50],
    10: [10, 20, 30, 40, 50]
}

# Iterate through the mapping
for k, threshold_values in k_threshold_mapping.items():
    for threshold in threshold_values:
        # Evaluate for head
        total_count_head, hit_count_head = tripleEvaluator.rerank(k=k, model=mlp_model, threshold=threshold, evaluate_for="head")
        head_percentage = hit_count_head / total_count_head * 100

        # Evaluate for tail
        total_count_tail, hit_count_tail = tripleEvaluator.rerank(k=k, model=mlp_model, threshold=threshold, evaluate_for="tail")
        tail_percentage = hit_count_tail / total_count_tail * 100

        # Calculate overall metrics
        overall_hit_count = hit_count_head + hit_count_tail
        overall_total_count = total_count_head + total_count_tail
        overall_percentage = (overall_hit_count / overall_total_count) * 100

        # Store the results in a dictionary
        results.append({
            "k": k,
            "threshold": threshold,
            "head": f"{head_percentage:.2f}%",
            "tail": f"{tail_percentage:.2f}%",
            "overall": f"{overall_percentage:.2f}%"
        })

# Print the results in a formatted way
for result in results:
    print(
        f"k={result['k']} | th={result['threshold']} | "
        f"head: {result['head']} | tail: {result['tail']} | overall: {result['overall']}"
    )

**XGB**

In [ ]:
results = []

# Define the mapping of k values to their respective threshold lists
k_threshold_mapping = {
    1: [1, 3, 5, 10, 20, 30, 40, 50],
    3: [3, 5, 10, 20, 30, 40, 50],
    5: [5, 10, 20, 30, 40, 50],
    10: [10, 20, 30, 40, 50]
}

# Iterate through the mapping
for k, threshold_values in k_threshold_mapping.items():
    for threshold in threshold_values:
        # Evaluate for head
        total_count_head, hit_count_head = tripleEvaluator.rerank(k=k, model=xgb_model, threshold=threshold, evaluate_for="head")
        head_percentage = hit_count_head / total_count_head * 100

        # Evaluate for tail
        total_count_tail, hit_count_tail = tripleEvaluator.rerank(k=k, model=xgb_model, threshold=threshold, evaluate_for="tail")
        tail_percentage = hit_count_tail / total_count_tail * 100

        # Calculate overall metrics
        overall_hit_count = hit_count_head + hit_count_tail
        overall_total_count = total_count_head + total_count_tail
        overall_percentage = (overall_hit_count / overall_total_count) * 100

        # Store the results in a dictionary
        results.append({
            "k": k,
            "threshold": threshold,
            "head": f"{head_percentage:.2f}%",
            "tail": f"{tail_percentage:.2f}%",
            "overall": f"{overall_percentage:.2f}%"
        })

# Print the results in a formatted way
for result in results:
    print(
        f"k={result['k']} | th={result['threshold']} | "
        f"head: {result['head']} | tail: {result['tail']} | overall: {result['overall']}"
    )

**LGBM**

In [ ]:
results = []

# Define the mapping of k values to their respective threshold lists
k_threshold_mapping = {
    1: [1, 3, 5, 10, 20, 30, 40, 50],
    3: [3, 5, 10, 20, 30, 40, 50],
    5: [5, 10, 20, 30, 40, 50],
    10: [10, 20, 30, 40, 50]
}

# Iterate through the mapping
for k, threshold_values in k_threshold_mapping.items():
    for threshold in threshold_values:
        # Evaluate for head
        total_count_head, hit_count_head = tripleEvaluator.rerank(k=k, model=lgbm_model, threshold=threshold, evaluate_for="head")
        head_percentage = hit_count_head / total_count_head * 100

        # Evaluate for tail
        total_count_tail, hit_count_tail = tripleEvaluator.rerank(k=k, model=lgbm_model, threshold=threshold, evaluate_for="tail")
        tail_percentage = hit_count_tail / total_count_tail * 100

        # Calculate overall metrics
        overall_hit_count = hit_count_head + hit_count_tail
        overall_total_count = total_count_head + total_count_tail
        overall_percentage = (overall_hit_count / overall_total_count) * 100

        # Store the results in a dictionary
        results.append({
            "k": k,
            "threshold": threshold,
            "head": f"{head_percentage:.2f}%",
            "tail": f"{tail_percentage:.2f}%",
            "overall": f"{overall_percentage:.2f}%"
        })

# Print the results in a formatted way
for result in results:
    print(
        f"k={result['k']} | th={result['threshold']} | "
        f"head: {result['head']} | tail: {result['tail']} | overall: {result['overall']}"
    )

**SCKIT LEARN ENSEMBLE**

In [ ]:
results = []

# Define the mapping of k values to their respective threshold lists
k_threshold_mapping = {
    1: [1, 3, 5, 10, 20, 30, 40, 50],
    3: [3, 5, 10, 20, 30, 40, 50],
    5: [5, 10, 20, 30, 40, 50],
    10: [10, 20, 30, 40, 50]
}

# Iterate through the mapping
for k, threshold_values in k_threshold_mapping.items():
    for threshold in threshold_values:
        # Evaluate for head
        total_count_head, hit_count_head = tripleEvaluator.rerank(k=k, model=ensemble_model, threshold=threshold, evaluate_for="head")
        head_percentage = hit_count_head / total_count_head * 100

        # Evaluate for tail
        total_count_tail, hit_count_tail = tripleEvaluator.rerank(k=k, model=ensemble_model, threshold=threshold, evaluate_for="tail")
        tail_percentage = hit_count_tail / total_count_tail * 100

        # Calculate overall metrics
        overall_hit_count = hit_count_head + hit_count_tail
        overall_total_count = total_count_head + total_count_tail
        overall_percentage = (overall_hit_count / overall_total_count) * 100

        # Store the results in a dictionary
        results.append({
            "k": k,
            "threshold": threshold,
            "head": f"{head_percentage:.2f}%",
            "tail": f"{tail_percentage:.2f}%",
            "overall": f"{overall_percentage:.2f}%"
        })

# Print the results in a formatted way
for result in results:
    print(
        f"k={result['k']} | th={result['threshold']} | "
        f"head: {result['head']} | tail: {result['tail']} | overall: {result['overall']}"
    )